In [2]:
from matplotlib import pyplot
from keras.datasets import cifar10
import numpy as np 
import pandas as pd

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]* X_train.shape[2]* X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2] * X_test.shape[3])
y_train = y_train.reshape(y_train.shape[0], )
y_test = y_test.reshape(y_test.shape[0], )

In [10]:
def train_val_split(data, labels, val_ratio):
    shuffled_indices = np.random.permutation(len(data))
    val_size = int(len(data) * val_ratio)
    val_indices = shuffled_indices[:val_size]
    train_indices = shuffled_indices[val_size:]
    return (data[train_indices], labels[train_indices], data[val_indices], labels[val_indices])

In [11]:
X_train, y_train,  X_val, y_val = train_val_split(X_train, y_train, .10)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(45000, 3072)
(10000, 3072)
(5000, 3072)
(45000,)
(10000,)
(5000,)
